In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.3 pyspark-shell'

import pyspark 
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

sc = SparkContext()
sc.setLogLevel("WARN")
spark = SparkSession(sc)


df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers","10.10.139.63:9092") \
    .option("subscribe", "calculated") \
    .option("startingOffsets", "earliest") \
    .load()

schema = StructType([
    StructField("visitor_platform", StringType()),
    StructField("ts_ingest", TimestampType()),
    StructField("article_title", StringType()),
    StructField("visitor_country", StringType()),
    StructField("visitor_os", StringType()),
    StructField("article", StringType()),
    StructField("visitor_browser", StringType()),
    StructField("visitor_page_timer", IntegerType()),
    StructField("visitor_page_height", IntegerType()),
])

print(df.schema)

dfs = df.selectExpr("CAST(value AS STRING)") \
      .select(from_json(col("value"), schema) \
      .alias("clicks"))

df_data = dfs.select("clicks.*")

df_data_grouped = (
    df_data
#         .withWatermark("timestamp", "20 second") # Late data?
        .groupBy(
            df_data['article_title'],
            window(df_data['ts_ingest'], "10 seconds"))
        .count()[]        
)


# Debug in terminal
# Docs output modes https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#output-modes
query = df_data_grouped.writeStream.outputMode("output").option("truncate", "false").format("console").start()
query.awaitTermination()